## 1. Import packages

In [23]:
#!pip install tweepy

In [24]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [25]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [26]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [27]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [28]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [29]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [06:57, 17.96it/s]


In [30]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [31]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7500/7500 [00:36<00:00, 204.72it/s]


In [32]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,"Mr. Toro Güero, #WearAMask #HighTransmissibility",Blocked by @RealJamesWoods,*Blunt* #Agave #advocate #diffusist #tequila #...,2010-12-08 06:21:51,2461,1586,10362,False,2020-07-26 09:33:31,Bravo sir. You're fast becoming one of the wor...,"[Mexico, failedpresident, covid19]",Twitter Web App,False
0,Venus Destination,Mumbai,A one stop Travel Planner thay avails the cust...,2018-08-16 20:12:44,14,13,2,False,2020-07-26 09:33:31,Grab the last hour seats to Lucknow from Dubai...,None,Twitter for Android,False
0,ZWkt,France,Workout music motivation,2020-01-26 11:53:45,0,0,10,False,2020-07-26 09:33:24,Pourquoi les esclaves noir porter des masques?...,"[covid, Covid_19, COVIDIOTS]",Twitter Web App,False
0,Afrobodies,South Africa,South African recombinant alpaca antibody prod...,2015-08-20 19:10:38,195,412,66,False,2020-07-26 09:33:22,Report coming from the UK which may have impor...,[COVID19],Twitter for Android,False
0,Sauk River Review,RTs are not endorsements. Views expressed are ...,American imagination artists living in PNW. Ha...,2011-09-17 04:57:07,4033,4994,102013,False,2020-07-26 09:33:19,@orblivion @klara_sjo @tsaylor Maybe that’s wh...,None,Twitter Web App,False


## 5. Save the data

### 5.1. Read past data

In [33]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (17176, 13)


### 5.2. Merge past and present data

In [34]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 17176 all tweets: 24676


### 5.3. Drop duplicates

In [35]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (24676, 13)


### 5.4. Export the updated data

In [38]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)